In [437]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold

# Prepare data

## Load data

Data is saved in dataprep and is loaded. These files did not have a resampling done. Sample length is 200. The data form the WISDM database is used.

In [438]:
# Read wdws from folder. 
path = '/home/jgrietens/Documents/smartbeat/dl-track/conv-har-selfimpl/dataprep'
folder_name = '/wdws_200'
f = path + folder_name

wdws = [pd.read_csv(f+'/'+str(i),index_col = 0) for i in range(5199)]

In [439]:
#read y and ids
y_df = pd.read_csv('/home/jgrietens/Documents/smartbeat/dl-track/conv-har-selfimpl/dataprep/y_df_200',index_col = 0)
ids_df = pd.read_csv('/home/jgrietens/Documents/smartbeat/dl-track/conv-har-selfimpl/dataprep/ids_df_200',index_col = 0)


In [440]:
### Create 1 hot encoded vectors from the y values. 
onehot_encoder = OneHotEncoder(sparse=False)
y_onehot = onehot_encoder.fit_transform(y_df, y=None)
#y_onehot is a numpyarray with one hot encoded values. 


In [441]:
#X: Convert list of dataframes to list of numpy arrays
i=0
for wdw in wdws:
    wdws[i] = wdw.loc[:,'x':'z'].as_matrix()
    i = i + 1
wdws_numpy = np.asarray(wdws)

## Create training/test set

In [442]:
train_x, test_x, train_y, test_y = train_test_split(wdws_numpy, y_onehot, test_size=0.20, random_state=42)

# Build model

In [443]:
act = ["Walking","Jogging","Sitting","Standing","Upstairs","Downstairs"] 


In [444]:
# Define hyperparemeters
wdw_length = 200
nr_classes = 6

In [445]:
#with tf.device("/device:GPU:0"):


In [446]:
# Define hyperparemeters
wdw_length = 200
nr_classes = 6



graph1 = tf.Graph()

with graph1.as_default():
    """The model definition"""
  
    with tf.name_scope("placeholders"):
        x = tf.placeholder(tf.float64,shape = [None,wdw_length,3])
        y = tf.placeholder(tf.float64, shape = [None,nr_classes])
        lr = tf.placeholder(tf.float64)
    
    with tf.name_scope("flatten_xyz"):
        x_flat = tf.layers.Flatten()(x)
   
    with tf.name_scope("fullyconnected"):
        #fully connected layer
        logits = tf.contrib.layers.fully_connected(x_flat,nr_classes, activation_fn=tf.nn.relu,trainable = True)
        
    with tf.name_scope("output"):
        prob = tf.contrib.layers.softmax(logits)
        y_pred = tf.round(prob)
    
    with tf.name_scope("loss"):
        entropy = tf.losses.softmax_cross_entropy(onehot_labels = y,logits = logits)
        #sum all the contributions
        l = tf.reduce_sum(entropy)
        
    with tf.name_scope("accuracies"):
        accuracy_i = tf.metrics.accuracy(labels = y,predictions = y_pred)
        acc = tf.reduce_sum(accuracy_i)
        
    with tf.name_scope("optim"):
        train_op = tf.train.AdamOptimizer(lr).minimize(l)
        init=tf.global_variables_initializer()
        
    with tf.name_scope("summaries"):
        tf.summary.scalar("loss", l)
        tf.summary.scalar("accuracy", acc)
        merged = tf.summary.merge_all()
        




# Train model

In [447]:

def train_epoch(X,y_labels,batch_size,n_epochs,learning_rate,graph):
   # tf.global_variables_initializer()

    N = len(X) - batch_size
    step = 0
    #learning_rate_np = np.array([learning_rate])
    
    with tf.Session(graph= graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(init)
        for epoch in range(n_epochs):
            pos = 0
            while pos < N:
                batch_X = X[pos:pos+batch_size]
                batch_y = y_labels[pos:pos+batch_size]
                feed_dict = {x: batch_X, y: batch_y, lr: learning_rate } 
                _, summary, loss, accuracy = sess.run([train_op,merged, l, acc],feed_dict=feed_dict)
                print("epoch %d, step %d, loss: %f" %(epoch, step, loss))
                
                step += 1
                pos += batch_size

In [448]:
train_epoch(train_x,train_y,10,50,0.01,graph1)


FailedPreconditionError: Attempting to use uninitialized value accuracies/accuracy/total
	 [[Node: accuracies/accuracy/total/read = Identity[T=DT_FLOAT, _class=["loc:@accuracies/accuracy/total"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](accuracies/accuracy/total)]]
	 [[Node: loss/Sum/_9 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_98_loss/Sum", tensor_type=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'accuracies/accuracy/total/read', defined at:
  File "/home/jgrietens/anaconda3/envs/tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jgrietens/anaconda3/envs/tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jgrietens/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel/__main__.py", line 

# Run model